In [ ]:
from google.colab import files
import os

print("Please upload BOTH train_features.csv and test_features.csv:")
_ = files.upload()

# sanity check
for fn in ['train_features.csv','test_features.csv']:
    if not os.path.exists(fn):
        raise FileNotFoundError(f"{fn} not found")
print("Uploaded:", os.listdir())


Saving FINALtestUpdated.csv to FINALtestUpdated (7).csv
Saving FINALtrainUpdated.csv to FINALtrainUpdated (5).csv
Uploaded files: ['FINALtestUpdated (7).csv', 'FINALtrainUpdated (5).csv']
Using → FINALtrainUpdated (5).csv as TRAIN and FINALtestUpdated (7).csv as TEST
Train shape: (120000, 25)
Test  shape: (7600, 25)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences  # not actually used

# load
df_train = pd.read_csv('train_features.csv')
df_test  = pd.read_csv('test_features.csv')

# split X/y
drop_cols = ['label']
X_train = df_train.drop(columns=drop_cols).values[..., None]
y_train = LabelEncoder().fit_transform(df_train['label'])

X_test  = df_test .drop(columns=drop_cols).values[..., None]
y_test  = LabelEncoder().fit_transform(df_test['label'])

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test:  {X_test.shape},  y_test:  {y_test.shape}")


Classes found: [1 2 3 4]
X_train shape: (120000, 21, 1)
X_test  shape: (7600, 21, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    InputLayer, Conv1D, GlobalMaxPooling1D, Dense
)

num_feats  = X_train.shape[1]
num_classes = len(set(y_train))

model = Sequential([
    InputLayer(input_shape=(num_feats, 1)),
    Conv1D(32, 3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Num GPUs Available:  0
Built with CUDA: True
GPU supported: False


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=32,
    verbose=2
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Test accuracy: {acc:.4%}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 21, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 21, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 10, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 5, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,644 (190.02 KB)

 Trainable params: 48,196 (188.27 KB)

 Non-trainable params: 448 (1.75 KB)